In [0]:
import nltk
import numpy as np

In [4]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


True

In [0]:
from nltk.corpus import gutenberg

In [6]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [0]:
text = ''
for txt in gutenberg.fileids():
  if 'shakespeare' in txt:
    text += gutenberg.raw(txt).lower()
chars = sorted(list(set(text)))

In [0]:
char_indices = dict((c,i) for i,c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

In [16]:
print('corpus length: {} total chars: {}'.format(len(text), len(chars)))

corpus length: 375542 total chars: 50


In [17]:
print(text[:500])

[the tragedie of julius caesar by william shakespeare 1599]


actus primus. scoena prima.

enter flauius, murellus, and certaine commoners ouer the stage.

  flauius. hence: home you idle creatures, get you home:
is this a holiday? what, know you not
(being mechanicall) you ought not walke
vpon a labouring day, without the signe
of your profession? speake, what trade art thou?
  car. why sir, a carpenter

   mur. where is thy leather apron, and thy rule?
what dost thou with thy best apparrell on


In [18]:
max_len = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - max_len, 3):
  sentences.append(text[i: i+ max_len])
  next_chars.append(text[i+max_len])
print('sequences : ', len(sentences))

sequences :  125168


In [23]:
sentences[0]

'[the tragedie of julius caesar by willia'

In [0]:
X = np.zeros((len(sentences), max_len, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)

In [0]:
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    X[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1
  

In [67]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
model = Sequential()
model.add(LSTM(256, input_shape=(max_len, len(chars))))
# model.add(LSTM(256))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.00001)
model.compile(loss='categorical_crossentropy', optimizer = optimizer)
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 256)               314368    
_________________________________________________________________
dense_14 (Dense)             (None, 50)                12850     
_________________________________________________________________
activation_14 (Activation)   (None, 50)                0         
Total params: 327,218
Trainable params: 327,218
Non-trainable params: 0
_________________________________________________________________


In [68]:
epochs = 6
batch_size = 128
model_structure = model.to_json()
with open("shakes_lstm2_model.json", "w") as json_file:
  json_file.write(model_structure)
for i in range(5):
  model.fit(X, y, batch_size=batch_size, epochs=epochs)
  model.save_weights("shakes_lstm2_weights_{}.h5".format(i+1))

Epoch 1/6
125168/125168 [==============================] - 118s 946us/step - loss: 3.4504
Epoch 2/6
125168/125168 [==============================] - 115s 921us/step - loss: 3.0071
Epoch 3/6
125168/125168 [==============================] - 111s 886us/step - loss: 2.9880
Epoch 4/6
125168/125168 [==============================] - 122s 974us/step - loss: 2.9789
Epoch 5/6
125168/125168 [==============================] - 113s 903us/step - loss: 2.9708
Epoch 6/6
125168/125168 [==============================] - 115s 916us/step - loss: 2.9623
Epoch 1/6
125168/125168 [==============================] - 115s 917us/step - loss: 2.9533
Epoch 2/6
125168/125168 [==============================] - 110s 879us/step - loss: 2.9434
Epoch 3/6
125168/125168 [==============================] - 122s 973us/step - loss: 2.9325
Epoch 4/6
125168/125168 [==============================] - 116s 929us/step - loss: 2.9206
Epoch 5/6
125168/125168 [==============================] - 110s 882us/step - loss: 2.9084
Epoch 6/6


In [0]:
from keras.models import model_from_json
with open("/content/shakes_lstm2_model.json", "r") as json_file:
  json_string = json_file.read()
model = model_from_json(json_string)
model.load_weights('/content/shakes_lstm2_weights_5.h5')

In [0]:
import random
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds/np.sum(exp_preds)
  probas = np.random.multinomial(1, preds,1)  #multinomial(num_samples, probabilities_list, size)
  return np.argmax(probas)
  

In [58]:
import sys
start_index = random.randint(0, len(text) - max_len - 1)
for diversity in [0.2, 0.5, 1.0]:
  print()
  print('----- diversity:', diversity)
  generated = ''
  sentence = text[start_index: start_index + max_len]
  generated += sentence
  print('----- Generating with seed: "' + sentence + '"')
  sys.stdout.write(generated)
  for i in range(400):
    x = np.zeros((1, max_len, len(chars)))
    for t, char in enumerate(sentence):
      x[0, t, char_indices[char]] = 1.
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]
    generated += next_char
    sentence = sentence[1:] + next_char # Ignoring 0th place character to make room for newly predicted character to be sent back to network for next character prediction.
    sys.stdout.write(next_char) 
    sys.stdout.flush() # flush internal buffer to the console so character appears immediately.
print()


----- diversity: 0.2
----- Generating with seed: "i beare my selfe;
(as i perchance heerea"
i beare my selfe;
(as i perchance heerea ee

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 e eeee ee    aeaee r ae ee e eeeee  aee e  ee  e e  a ee re  a aeeea  

KeyboardInterrupt: ignored